# Data Prep for MultiModal Modeling

## Author: Ian Scarff (iscarff123)

In [1]:
import pandas as pd
import numpy as np
import json
import os
import imageio
import random
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
import time
import datetime
from hatesonar import Sonar # This is the hate speech detection library; it is based on bert
from transformers import pipeline

## Load in reference data.

In [2]:
### Load in reference data

with open('training_data_task3.txt') as f: ### Training data
    training = json.load(f)
    
with open('testing_data_task3.txt') as f: ### Testing data
    testing = json.load(f)
    
classes = ['Smears', 'Loaded Language', 'Name calling/Labeling', 'Glittering generalities (Virtue)',
               'Appeal to (Strong) Emotions', 'Appeal to fear/prejudice', 'Transfer', 'Doubt',
               'Exaggeration/Minimisation', 'Whataboutism', 'Slogans', 'Flag-waving',
               "Misrepresentation of Someone's Position (Straw Man)", 'Causal Oversimplification',
               'Thought-terminating cliché', 'Black-and-white Fallacy/Dictatorship', 'Appeal to authority',
               'Reductio ad hitlerum', 'Repetition', 'Obfuscation, Intentional vagueness, Confusion',
               'Presenting Irrelevant Data (Red Herring)', 'Bandwagon']

### Create Class Binarizer
one_hot = MultiLabelBinarizer()
one_hot.fit([classes])

MultiLabelBinarizer()

In [3]:
one_hot.classes_

array(['Appeal to (Strong) Emotions', 'Appeal to authority',
       'Appeal to fear/prejudice', 'Bandwagon',
       'Black-and-white Fallacy/Dictatorship',
       'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation',
       'Flag-waving', 'Glittering generalities (Virtue)',
       'Loaded Language',
       "Misrepresentation of Someone's Position (Straw Man)",
       'Name calling/Labeling',
       'Obfuscation, Intentional vagueness, Confusion',
       'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum',
       'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché',
       'Transfer', 'Whataboutism'], dtype=object)

In [4]:
training[0:3]

[{'id': '182',
  'labels': ['Exaggeration/Minimisation',
   'Name calling/Labeling',
   'Smears',
   'Transfer'],
  'text': 'The most costly errors in all of history\n\nWorld Health Organization (WHO)\nPreliminary investigations conducted by the Chinese authorities have found no clear evidence of human-to-human transmission of the novel #coronavirus (2019-nCoV) identified in #Wuhan, #China\n14 Jan 2020\n\nWORLD NEWS FEBRUARY 3, 2020 / 10:33 PM / 2 MONTHS AGO\nWHO chief says widespread travel bans not needed to beat China virus\n',
  'image': '182_image.png'},
 {'id': '366_batch_2',
  'labels': ['Causal Oversimplification',
   'Loaded Language',
   'Name calling/Labeling'],
  'text': 'MY PARENTS WERE KILLED AS A RESULT OF A GUN FREE ZONE.\n\nI CARRIED A GUN AT ALL TIMES TILL IT WAS MADE A FELONY TO CARRY IN CERTAIN AREAS. \nA MAN WITH A GUN STARTED SHOOTING PEOPLE...\nI HAD THE PERFECT SHOT BUT WAS DISARMED BY LAWS.\n\n',
  'image': '366_image_batch_2.png'},
 {'id': '148',
  'labels': [

In [5]:
len(training)

727

In [6]:
testing[0:3]

[{'id': '566_batch_2',
  'labels': ['Appeal to (Strong) Emotions', 'Smears'],
  'text': "A FRIENDLY REMINDER...\n\nFERGUSON\n\nBALTIMORE\n\nMILWAUKEE\n\nCHARLOTTE\n\nIN CASE Y'ALL FORGOT !\n",
  'image': '566_image_batch_2.png'},
 {'id': '738_batch_2',
  'labels': ['Black-and-white Fallacy/Dictatorship',
   'Flag-waving',
   'Loaded Language',
   'Name calling/Labeling'],
  'text': "We're fed up.\nWe're not asking for our country back.\nWe're TAKING IT BACK\nWE THE PEOPLE ARE COMING\nAmerica was founded by pissed off people.\nAnd they're pissed again!",
  'image': '738_image_batch_2.png'},
 {'id': '885_batch_2',
  'labels': ['Smears'],
  'text': "PRESIDENTS' DAY SALE\n\nEVERYONE MUST GO",
  'image': '885_image_batch_2.png'}]

In [7]:
len(testing)

187

In [8]:
len(training) + len(testing)

914

## Extract Image Names, Text, & Labels

### Training data

In [9]:
image_name_train = []
text_train = []
one_hot_labels_train = [] ### List to hold one hot labels

for obsv in training: ### Go through the training data

    ### Images ###
    image_name_train.append(obsv['image'])
    
    text_train.append(obsv['text'])
    
    ### Labels
    one_hot_labels_train.append(one_hot.transform([obsv['labels']])[0])


In [10]:
image_name_train[0:3]

['182_image.png', '366_image_batch_2.png', '148_image.png']

In [11]:
text_train[0:3]

['The most costly errors in all of history\n\nWorld Health Organization (WHO)\nPreliminary investigations conducted by the Chinese authorities have found no clear evidence of human-to-human transmission of the novel #coronavirus (2019-nCoV) identified in #Wuhan, #China\n14 Jan 2020\n\nWORLD NEWS FEBRUARY 3, 2020 / 10:33 PM / 2 MONTHS AGO\nWHO chief says widespread travel bans not needed to beat China virus\n',
 'MY PARENTS WERE KILLED AS A RESULT OF A GUN FREE ZONE.\n\nI CARRIED A GUN AT ALL TIMES TILL IT WAS MADE A FELONY TO CARRY IN CERTAIN AREAS. \nA MAN WITH A GUN STARTED SHOOTING PEOPLE...\nI HAD THE PERFECT SHOT BUT WAS DISARMED BY LAWS.\n\n',
 "VOTE IT OUT\nVOTE IT OUT\n\nHE SAID IT WASN'T REAL.\nHE SAID IT'S CONTAINED.\nHE SAID IT WOULD DISAPPEAR.\nHE BLAMED DEMOCRATS.\nHE SHARED HIS HUNCHES.\nHE OVERRULED HIS SCIENTISTS.\nHE HAS FAILED.\nHE HAS ENDANGERED\nEVERY ONE OF US.\n\nVOTE IT OUT\nVOTE IT OUT\n"]

In [12]:
one_hot_labels_train[0:3]

[array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])]

### Testing data

In [13]:
image_name_test = []
text_test = []
one_hot_labels_test = [] ### List to hold one hot labels

for obsv in testing: ### Go through the test data

    ### Images ###
    image_name_test.append(obsv['image'])
    
    text_test.append(obsv['text'])
    
    ### Labels
    one_hot_labels_test.append(one_hot.transform([obsv['labels']])[0])

In [14]:
image_name_test[0:3]

['566_image_batch_2.png', '738_image_batch_2.png', '885_image_batch_2.png']

In [15]:
text_test[0:3]

["A FRIENDLY REMINDER...\n\nFERGUSON\n\nBALTIMORE\n\nMILWAUKEE\n\nCHARLOTTE\n\nIN CASE Y'ALL FORGOT !\n",
 "We're fed up.\nWe're not asking for our country back.\nWe're TAKING IT BACK\nWE THE PEOPLE ARE COMING\nAmerica was founded by pissed off people.\nAnd they're pissed again!",
 "PRESIDENTS' DAY SALE\n\nEVERYONE MUST GO"]

In [16]:
one_hot_labels_test[0:3]

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])]

## Text Preprocessing

In [17]:
# Create an object of Sonar Hate Speech Detection
sonar = Sonar()

/home/ian/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/ian/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ian/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ian/.local/lib/python3.6/site-packages/skl

In [18]:
def hate_speech_classifier(text, Class):
    for i in text:
        sonar_dict = sonar.ping(text=i)
        Class.append(list(sonar_dict.values())[1])
        
def sentiment_classifier(text):
    sent = []
    for i in text:
        result = sentimentanalyzer(i)[0]
        sent.append(result['label'])
    return sent

In [19]:
sentimentanalyzer = pipeline("sentiment-analysis", device = 1)

### Training Data

#### Hate Speech

In [20]:
### Training data
hate_speech_class_train = []
hate_speech_classifier(text_train, hate_speech_class_train)

In [21]:
hate_speech_class_train[0:10]

['neither',
 'neither',
 'neither',
 'offensive_language',
 'neither',
 'hate_speech',
 'neither',
 'neither',
 'neither',
 'neither']

In [22]:
# 0 = neither, 1 = hate_speech, 2 = offensive_language

hate_speech_label_train = []

for i in hate_speech_class_train:
    if i == 'neither':
        hate_speech_label_train.append(0)
        
    elif i == 'hate_speech':
        hate_speech_label_train.append(1)
        
    elif i == 'offensive_language':
        hate_speech_label_train.append(2)

In [23]:
hate_speech_label_train[0:10]

[0, 0, 0, 2, 0, 1, 0, 0, 0, 0]

#### Sentiment

In [24]:
sentiment_train = sentiment_classifier(text_train)

In [25]:
set(sentiment_train)

{'NEGATIVE', 'POSITIVE'}

In [26]:
# 0 = Positive, 1 = Negative
sentiment_label_train = []
for i in sentiment_train:
    if i == 'NEGATIVE':
        sentiment_label_train.append(0)
    elif i == 'POSITIVE':
        sentiment_label_train.append(1)

In [27]:
sentiment_label_train[0:10]

[0, 0, 0, 1, 0, 1, 1, 0, 0, 0]

### Test Data

#### Hate Speech

In [28]:
### Training data
hate_speech_class_test = []
hate_speech_classifier(text_test, hate_speech_class_test)

In [29]:
hate_speech_class_test[0:10]

['neither',
 'neither',
 'neither',
 'neither',
 'neither',
 'neither',
 'neither',
 'neither',
 'neither',
 'offensive_language']

In [30]:
# 0 = neither, 1 = hate_speech, 2 = offensive_language

hate_speech_label_test = []

for i in hate_speech_class_test:
    if i == 'neither':
        hate_speech_label_test.append(0)
        
    elif i == 'hate_speech':
        hate_speech_label_test.append(1)
        
    elif i == 'offensive_language':
        hate_speech_label_test.append(2)

In [31]:
hate_speech_label_test[0:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]

#### Sentiment

In [32]:
sentiment_test = sentiment_classifier(text_test)

In [33]:
set(sentiment_test)

{'NEGATIVE', 'POSITIVE'}

In [34]:
# 0 = Positive, 1 = Negative
sentiment_label_test = []
for i in sentiment_test:
    if i == 'NEGATIVE':
        sentiment_label_test.append(0)
    elif i == 'POSITIVE':
        sentiment_label_test.append(1)

In [35]:
sentiment_label_test[0:10]

[1, 0, 1, 1, 1, 0, 0, 0, 0, 1]

## Assemble DataFrames

### Training

In [36]:
training_df = pd.DataFrame({'Image' : image_name_train,
                            'Text' : text_train,
                            'Hate' : hate_speech_label_train,
                            'Sentiment' : sentiment_label_train,
                            'Labels' : one_hot_labels_train})

In [37]:
training_df

,Image,Text,Hate,Sentiment,Labels
0,182_image.png,The most costly errors in all of history\n\nWo...,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."
1,366_image_batch_2.png,MY PARENTS WERE KILLED AS A RESULT OF A GUN FR...,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
2,148_image.png,VOTE IT OUT\nVOTE IT OUT\n\nHE SAID IT WASN'T ...,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,58_image_batch_2.png,IF YOU DONT LISTEN TO DESPACITO YOU AIN'T LATI...,2,1,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,816_image_batch_2.png,FAIR AND BALANCED\n\nFAUX NEWS\n\nWE DISTORT Y...,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...
722,687_image_batch_2.png,"Asked what the ""D.C."" stands for in ""Washingto...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
723,516_image_batch_2.png,BREAKING NEWS: The Chicago Police Dept has rep...,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
724,45_image_batch_2.png,IF WE GIVE UP EVERYTHING THAT OFFENDS SOMEONE ...,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
725,86_image_batch_2.png,"YOU'D THINK YOU WERE IN A WHORE HOUSE, SEEING ...",2,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."


### Testing

In [38]:
test_df = pd.DataFrame({'Image' : image_name_test,
                            'Text' : text_test,
                            'Hate' : hate_speech_label_test,
                            'Sentiment' : sentiment_label_test,
                            'Labels' : one_hot_labels_test})

In [39]:
test_df

,Image,Text,Hate,Sentiment,Labels
0,566_image_batch_2.png,A FRIENDLY REMINDER...\n\nFERGUSON\n\nBALTIMOR...,0,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,738_image_batch_2.png,We're fed up.\nWe're not asking for our countr...,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, ..."
2,885_image_batch_2.png,PRESIDENTS' DAY SALE\n\nEVERYONE MUST GO,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,41_image_batch_2.png,TRUDEAU'S PRIORITIES\n1.ENRICHING HIS FRIENDS ...,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, ..."
4,169_image.png,Our elders were called to war to save lives.\n...,0,1,"[0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
182,867_image_batch_2.png,"AT THIS POINT, SHOULDN'T HIS RALLIES BE CONSID...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
183,901_image_batch_2.png,HEY YOU GUYSSSS!\n\nI CAN TRAVEL NOW\nTHE VACC...,0,1,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
184,932_image_batch_2.png,"President Trump, infected with COVID-19, retur...",0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
185,967_image_batch_2.png,Your parades are cute!\n\nWait till you see ou...,0,1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."


## Save to JSON

In [40]:
training_df.to_json('MultiModal_training_data.json')
test_df.to_json('MultiModal_testing_data.json')

In [41]:
pd.read_json('MultiModal_training_data.json')

,Image,Text,Hate,Sentiment,Labels
0,182_image.png,The most costly errors in all of history\n\nWo...,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."
1,366_image_batch_2.png,MY PARENTS WERE KILLED AS A RESULT OF A GUN FR...,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
2,148_image.png,VOTE IT OUT\nVOTE IT OUT\n\nHE SAID IT WASN'T ...,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,58_image_batch_2.png,IF YOU DONT LISTEN TO DESPACITO YOU AIN'T LATI...,2,1,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,816_image_batch_2.png,FAIR AND BALANCED\n\nFAUX NEWS\n\nWE DISTORT Y...,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...
722,687_image_batch_2.png,"Asked what the ""D.C."" stands for in ""Washingto...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
723,516_image_batch_2.png,BREAKING NEWS: The Chicago Police Dept has rep...,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
724,45_image_batch_2.png,IF WE GIVE UP EVERYTHING THAT OFFENDS SOMEONE ...,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
725,86_image_batch_2.png,"YOU'D THINK YOU WERE IN A WHORE HOUSE, SEEING ...",2,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."


In [42]:
pd.read_json('MultiModal_training_data.json').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727 entries, 0 to 726
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Image      727 non-null    object
 1   Text       727 non-null    object
 2   Hate       727 non-null    int64 
 3   Sentiment  727 non-null    int64 
 4   Labels     727 non-null    object
dtypes: int64(2), object(3)
memory usage: 34.1+ KB
